In [1]:
# Many thanks for Yu-Cheng Chen
# This code is modified from https://github.com/s885219s/flx-gaze-eye-cropper/blob/master/DIRL_eye_cropper_silver.ipynb

In [2]:
import sys
import os
import dlib
import glob
import math
import cv2
import numpy as np
import numpy as np
import pickle
import _pickle as cPickle
import matplotlib.pyplot as plt

In [3]:
def input2data(eye_rc, eye_lc, FP_seq, img, pos = None):
#     print("Processing file: {}".format(os.path.basename(img)))
    frame = cv2.imread(img)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    eye_cx = (eye_rc[0]+eye_lc[0])*0.5
    eye_cy = (eye_rc[1]+eye_lc[1])*0.5
    eye_center = [eye_cx, eye_cy]
    eye_len = np.absolute(eye_rc[0]-eye_lc[0])
    bx_d5w = eye_len*3/4
    bx_h = 1.5*bx_d5w
    sft_up = bx_h*7/12
    sft_low = bx_h*5/12
    img_eye = frame[int(eye_cy-sft_up+0.5):int(eye_cy+sft_low+0.5),int(eye_cx-bx_d5w+0.5):int(eye_cx+bx_d5w+0.5)]
    ori_size = [img_eye.shape[0],img_eye.shape[1]]
    LT_coor = [int(eye_cy-sft_up), int(eye_cx-bx_d5w)] # (y,x)
    img_eye = cv2.resize(img_eye, (new_size[1], new_size[0]))
    
    origin_xy = np.array(LT_coor)
    origin_xy = origin_xy[::-1]
    ratio = [new_size[1]/ori_size[1], new_size[0]/ori_size[0]]
    ratio = np.array(ratio)
    # create anchor maps
    ach_map = []
    ap = np.zeros(14, dtype=np.int32)
    for i,d in enumerate(FP_seq):
        ap[i*2] = d[0]
        ap[i*2+1] = d[1]
        resize_x = int((d[0]-LT_coor[1])*new_size[1]/ori_size[1] +0.5)
        resize_y = int((d[1]-LT_coor[0])*new_size[0]/ori_size[0] +0.5)
        # y
        ach_map_y = np.expand_dims(np.expand_dims(np.arange(0, new_size[0]) - resize_y, axis=1), axis=2)
        ach_map_y = np.tile(ach_map_y, [1,new_size[1],1])
        #print(ach_map_y[:,:,0])
        # x
        ach_map_x = np.expand_dims(np.expand_dims(np.arange(0, new_size[1]) - resize_x, axis=0), axis=2)
        ach_map_x = np.tile(ach_map_x, [new_size[0],1,1])
        #print(ach_map_x[:,:,0])
        if (i ==0):
            ach_map = np.concatenate((ach_map_x, ach_map_y), axis=2)
        else:
            ach_map = np.concatenate((ach_map, ach_map_x, ach_map_y), axis=2)
            
    for i in range(6):
        try:
            anchor = [int(np.where(ach_map[0,:,2*i+0] ==0)[0]),int(np.where(ach_map[:,0,2*i+1] ==0)[0])]
        except:
            print('Error: ',os.path.basename(img))      
#     cv2.imwrite(output_folder_path+'test_'+pos+'.png',img_eye)
    return os.path.basename(img), img_eye, ach_map, ap, origin_xy, ratio

def output_pickle(pickle_list,pos = None):
    d ={'img':[],'anchor_map':[],'p':[],'v':[],'h':[],'ap':[],'origin_xy':[],'ratio':[]}
    user = pickle_list[0][0][:4]
    pose = pickle_list[0][0][5:pickle_list[0][0].find('P')] 
    for p in pickle_list:
        fn_name = p[0][:-4]
        d['img'].append(p[1])
        d['anchor_map'].append(p[2])
        d['p'].append(int((fn_name.split("_")[1]).split("P")[0]))
        d['h'].append(int((fn_name.split("_")[2]).split("H")[0]))
        d['v'].append(int((fn_name.split("_")[3]).split("V")[0]))  
        d['ap'].append(p[3])
        d['origin_xy'].append(p[4])
        d['ratio'].append(p[5])
        
    new_path = os.path.join(save_pickle_path,pose,user)
    if not os.path.exists(new_path):
        os.makedirs(new_path)
    f = open(new_path + '/' + user + '_' + pos, 'wb')
    pickle.dump(d, f)
    f.close()               
    print(new_path,'Completed!')
#     plt.figure()
#     plt.imshow(pickle_list[0][1])
#     plt.show()
    return

In [4]:
dataset_dir = './'
tar_pose = '0'
image_width = 64
image_height = 48
feature_point_num = 7

In [5]:
new_size = [image_height,image_width]
img_list = glob.glob(dataset_dir + 'images/'+ tar_pose +'/00*/*.png')
txt_list = glob.glob(dataset_dir + 'labels/'+ tar_pose +'/00*/*.txt')
save_pickle_path = dataset_dir + 'training_inputs'
if not os.path.isdir(save_pickle_path):
    os.makedirs(save_pickle_path)
img_list.sort()
txt_list.sort()
R_FP_seq = np.zeros((feature_point_num,2), dtype='int32')
L_FP_seq = np.zeros((feature_point_num,2), dtype='int32')
R_pickle = []
L_pickle = []
temp = os.path.basename(img_list[0])[:7]
for (img,txt) in zip(img_list,txt_list):
    if not temp == os.path.basename(img)[:7]:
        output_pickle(R_pickle,pos = 'R')
        output_pickle(L_pickle,pos = 'L')
        R_pickle = []
        L_pickle = []
    temp = os.path.basename(img)[:7]
    eye_feature_point = np.genfromtxt(txt , delimiter=',') 
    R_FP_seq = eye_feature_point[:7,2:4]
    R_eye_rc = (eye_feature_point[0,2],eye_feature_point[0,3])
    R_eye_lc = (eye_feature_point[3,2],eye_feature_point[3,3])
    L_FP_seq = eye_feature_point[7:14,2:4]
    L_eye_rc = (eye_feature_point[10,2],eye_feature_point[10,3])
    L_eye_lc = (eye_feature_point[7,2],eye_feature_point[7,3])
    R_pickle.append(input2data(R_eye_rc,R_eye_lc,R_FP_seq,img,pos = 'R'))
    L_pickle.append(input2data(L_eye_rc,L_eye_lc,L_FP_seq,img,pos = 'L'))
output_pickle(R_pickle,pos = 'R')
output_pickle(L_pickle,pos = 'L')

./training_inputs\0\0000 Completed!
./training_inputs\0\0000 Completed!
./training_inputs\0\0001 Completed!
./training_inputs\0\0001 Completed!
